# Eco Score Pipeline for beautofuel

In [60]:
%load_ext autoreload
%autoreload 2

from influxdb import DataFrameClient

from lib.utils.constants import INFLUXDB_HOST, INFLUXDB_PORT, INFLUXDB_USER, INFLUXDB_PASSWORD, INFLUXDB_DB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Pipeline configuration

In [61]:
# Database client
grafanadb = DataFrameClient(
    host=INFLUXDB_HOST,
    port=INFLUXDB_PORT,
    database=INFLUXDB_DB,
    username=INFLUXDB_USER,
    password=INFLUXDB_PASSWORD
)

user = "zlatka"

## Helper functions

In [62]:
def get_query_result_value(result, field):
    return result['tracks'][field][0]

def get_query_end(phase):
    return "FROM \"tracks\" WHERE (\"phase\"='{}' AND \"user\"='{}')".format(phase, user)

## Setup Queries

In [63]:
# Consumption queries
stdddev_consumption_query = "SELECT stddev(\"consumption\") {};".format(get_query_end(1))
mean_consumption_query = "SELECT mean(\"consumption\") {};".format(get_query_end(1))
min_consumption_query = "SELECT min(\"consumption\") {};".format(get_query_end(1))
max_consumption_query = "SELECT max(\"consumption\") {};".format(get_query_end(1))

## Execute Queries

In [64]:
# Execute consumption queries
stddev_consumption = get_query_result_value(grafanadb.query(stdddev_consumption_query), 'stddev')
mean_consumption = get_query_result_value(grafanadb.query(mean_consumption_query), 'mean')
min_consumption = get_query_result_value(grafanadb.query(min_consumption_query), 'min')
max_consumption = get_query_result_value(grafanadb.query(max_consumption_query), 'max')

In [65]:
# Print consumption queries results
print("consumption_100_std:", stddev_consumption)
print("consumption_100_mean:", mean_consumption)
print("consumption_100_min:", min_consumption)
print("consumption_100_max:", max_consumption)

consumption_100_std: 1.5725679635551524
consumption_100_mean: 6.581999999999999
consumption_100_min: 4.65
consumption_100_max: 8.51


## Eco-score Calculation

In [66]:
tracks_2_query = "SELECT \"consumption\" {};".format(get_query_end(2))

tracks_2 = grafanadb.query(tracks_2_query)

track_index = 1

for _, track_row in tracks_2['tracks'].iterrows():
    track_2_consumption = track_row['consumption']

    track_2_eco_score = None

    lower_consumption_limit = mean_consumption - stddev_consumption
    upper_consumption_limit = mean_consumption + stddev_consumption

    if track_2_consumption >= upper_consumption_limit:
        eco_score = 0
    elif track_2_consumption <= lower_consumption_limit:
        eco_score = 100
    else:
        track_2_consumption = track_2_consumption - lower_consumption_limit
        upper_consumption_limit = upper_consumption_limit - lower_consumption_limit

        eco_score = int((track_2_consumption / upper_consumption_limit) * 100)

    print("{}. eco_score: {}".format(track_index, eco_score))
    track_index += 1

1. eco_score: 100
2. eco_score: 75
3. eco_score: 9
4. eco_score: 0
5. eco_score: 65
6. eco_score: 74
7. eco_score: 15
8. eco_score: 0
